In [1]:
import re
import numpy as np
import pandas as pd
from unidecode import unidecode

In [2]:
# 从txt中获取数据并查看长度
ontology = []
for line in open('dbpedia_2016-04.txt', "r",encoding='UTF-8'):
    ontology.append(line)
len(ontology)

30793

In [3]:
o2 = []
for j in ontology:
    j = unidecode(j)
    if '#subClassOf>' in j:
        result1 = re.search('/[^/#]*>',j)
        start_name = []
        if result1:
            start_name.append(result1.group(0))
        else:
            continue
        result2 = re.findall('[/|#]+[^/#]+>',j)
        result2 = result2[1:]
        o2.append(start_name+result2)

In [4]:
df2 = pd.DataFrame(o2, columns = ['start_name','relationship','end_name'])

In [5]:
# 删除多余的符号
df2['start_name'] = df2['start_name'].str.replace('[/>#]+', '')

df2['end_name'] = df2['end_name'].str.replace('[/>#]+', '')

df2['relationship'] = df2['relationship'].str.replace('[/>#]+', '')

print(len(df2))
df2.head()

763


,start_name,relationship,end_name
0,BasketballLeague,subClassOf,SportsLeague
1,NaturalEvent,subClassOf,Event
2,Province,subClassOf,GovernmentalAdministrativeRegion
3,LunarCrater,subClassOf,Crater
4,MotorsportSeason,subClassOf,SportsSeason


In [6]:
# from collections import Counter
# Counter(df2['end_name']).most_common(100)

In [10]:
df2.to_csv('dict.csv')

In [8]:
### 在Excel里对dict.csv进行改变：将想保留的标签对应的母标签变为它本身，这样迭代替换时还是它自己

In [11]:
d = pd.read_csv('dict.csv',encoding = 'utf-8')
d

,Unnamed: 0,start_name,relationship,end_name
0,0,BasketballLeague,subClassOf,SportsLeague
1,1,NaturalEvent,subClassOf,Event
2,2,Province,subClassOf,GovernmentalAdministrativeRegion
3,3,LunarCrater,subClassOf,Crater
4,4,MotorsportSeason,subClassOf,SportsSeason
...,...,...,...,...
758,758,LegalCase,subClassOf,Case
759,759,Crustacean,subClassOf,Animal
760,760,Broadcaster,subClassOf,Organisation
761,761,MusicDirector,subClassOf,MusicalArtist


In [27]:
cla_dict = d.set_index('start_name')['end_name'].to_dict()

In [28]:
len(cla_dict)

755

In [29]:
entity = pd.read_csv('entity.csv',encoding = 'utf-8')

In [30]:
entity = entity.drop('Unnamed: 0', axis = 1)
entity['Class'] = entity[':LABEL']
entity

,entityName,entityId:ID,entityProperty,:LABEL,Class
0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb...",Election,Election
1,New Road Team,1,"name: New Road, name: New Road Team, nick: N...",SoccerClub,SoccerClub
2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0...",Album,Album
3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ...",Plant,Plant
4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa...",Person,Person
...,...,...,...,...,...
4691087,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo...",Ship,Ship
4691088,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind...",Person,Person
4691089,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi...",Person,Person
4691090,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa...",CareerStation,CareerStation


In [31]:
# 第五次转换
# 树的高度是5，所以转换五次
entity['Class'] = entity['Class'].map(cla_dict)
entity['Class'] = entity['Class'].map(cla_dict)
entity['Class'] = entity['Class'].map(cla_dict)
entity['Class'] = entity['Class'].map(cla_dict)
entity['Class'] = entity['Class'].map(cla_dict)
entity

,entityName,entityId:ID,entityProperty,:LABEL,Class
0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb...",Election,Event
1,New Road Team,1,"name: New Road, name: New Road Team, nick: N...",SoccerClub,Organisation
2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0...",Album,Work
3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ...",Plant,Species
4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa...",Person,Person
...,...,...,...,...,...
4691087,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo...",Ship,Transportation
4691088,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind...",Person,Person
4691089,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi...",Person,Person
4691090,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa...",CareerStation,TimePeriod


In [32]:
set(entity['Class'])

{'Activity',
 'Award',
 'Device',
 'Event',
 'Food',
 'Organisation',
 'Person',
 'Place',
 'Species',
 'Thing',
 'TimePeriod',
 'Transportation',
 'Work',
 nan}

In [33]:
a = entity[entity['Class'].isnull()]
a

,entityName,entityId:ID,entityProperty,:LABEL,Class
1104398,Alpine A110 1300 <<Super,1370761,"_S__1: name A110 1300 <<Super, _S__1: powerOu...",type,NaN


In [63]:
en_new = entity
print(len(en_new))
en_new = en_new.dropna()
print(len(en_new))
en_new = en_new.rename(columns = {'Class':'Home_Label',':LABEL':'entityType'})
en_new

4691092
4691091


,entityName,entityId:ID,entityProperty,entityType,Home_Label
0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb...",Election,Event
1,New Road Team,1,"name: New Road, name: New Road Team, nick: N...",SoccerClub,Organisation
2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0...",Album,Work
3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ...",Plant,Species
4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa...",Person,Person
...,...,...,...,...,...
4691087,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo...",Ship,Transportation
4691088,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind...",Person,Person
4691089,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi...",Person,Person
4691090,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa...",CareerStation,TimePeriod


In [48]:
labs = list(set(en_new['Home_Label']))
labs

['Event',
 'Organisation',
 'TimePeriod',
 'Activity',
 'Transportation',
 'Thing',
 'Food',
 'Place',
 'Species',
 'Person',
 'Device',
 'Work',
 'Award']

In [59]:
for i in labs:
    df = en_new[en_new['Home_Label']==i]
    df = df.drop('Home_Label', axis = 1)
    df.to_csv('{name}.csv'.format(name=i))

In [64]:
en_new = en_new.rename(columns = {'Home_Label':':LABEL'})
en_new

,entityName,entityId:ID,entityProperty,entityType,:LABEL
0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb...",Election,Event
1,New Road Team,1,"name: New Road, name: New Road Team, nick: N...",SoccerClub,Organisation
2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0...",Album,Work
3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ...",Plant,Species
4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa...",Person,Person
...,...,...,...,...,...
4691087,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo...",Ship,Transportation
4691088,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind...",Person,Person
4691089,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi...",Person,Person
4691090,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa...",CareerStation,TimePeriod


In [65]:
en_new.to_csv('entity_new.csv')